In [1]:
import psycopg2
import geopandas as gpd
from pathlib import Path
import json

In [2]:
credentials = Path('../../inputs/db_credentials.json')
with open(credentials, 'r') as f:
    credentials = json.load(f)

user     = credentials['user']
password = credentials['password']
host     = credentials['host']
port     = credentials['port']
database = credentials['database']

In [15]:
# query = f'SELECT ST_Transform(ST_Buffer(ST_UNION(geom), 0), {SRID}) as geom FROM thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;
# query = f'SELECT ST_Transform(ST_ExteriorRing(ST_UNION(geom)), {SRID}) as geom FROM thuc_cell_geo({lat}, {lon}, thuc_near_run_id({lon},{lat}))' ;


# cur.execute("SELECT table_name FROM information_schema.tables")

# query = f'SELECT ST_AsGeoJSON(ST_Transform(ST_UNION(ST_ExteriorRing(geom)), {SRID})) FROM thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;

# query = f'SELECT geom FROM thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;
# query = f'SELECT ST_Transform(ST_UNION(ST_ExteriorRing(geom)), {SRID}) FROM thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;


# query = f'SELECT thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;
# query = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE'"

In [5]:
# conn = psycopg2.connect(dbname='THUC', user='dbadmin', password='vxn&584dkj+%38#3aj!', host="130.74.75.55", port="5432")
conn = psycopg2.connect(database=database,
                        host=host,
                        user=user,
                        password=password,
                        port=port)
cur = conn.cursor()

SRID = 4326
lon, lat = -89.9078220, 34.2385274

# query = f'SELECT ST_AsGeoJSON(ST_UNION(ST_Transform(ST_ExteriorRing(geom), {SRID}))) FROM thuc_cell_geo({lon}, {lat}, thuc_near_run_id({lon},{lat}))' ;
# boundary_sql = f"SELECT ST_AsGeoJSON(ST_Transform(ST_UNION(geom), {SRID})) FROM thuc_cell_geo({lon},{lat}, thuc_near_run_id({lon},{lat}))"
# cells_sql = f"SELECT ST_AsGeoJSON(ST_Transform(ST_UNION(ST_ExteriorRing(geom)), {SRID})) FROM thuc_cell_geo({lon},{lat},thuc_near_run_id({lon},{lat}))"


simple_cells = "SELECT * FROM thuc_1002_annagnps_cell_ids LIMIT 10"

query = simple_cells

cur.execute(query)

tables = cur.fetchall()

for table in tables:
    print(table[0])

cur.close()
conn.close()

1
2
3
4
5
6
7
8
9
10


In [7]:
from sqlalchemy import URL, create_engine, text as sql_text
import geopandas as gpd

# create a SQLAlchemy engine object
url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

engine = create_engine(url_object)

# query = "SELECT * FROM thuc_1002_annagnps_cell_ids LIMIT 5"
query = "SELECT * FROM thuc_1148_annagnps_cell_ids WHERE dn ~ '4$' LIMIT 5"
# query = "SELECT * FROM thuc_1002_annagnps_cell_data_section ORDER BY cell_id"# WHERE cell_id=22"

gdf = gpd.read_postgis(sql=sql_text(query), con=engine.connect(), geom_col='geom')
# gdf = gpd.read_postgis(sql=sql_text(query), con=conn, geom_col='geom')
gdf

ProgrammingError: (psycopg2.errors.UndefinedFunction) operator does not exist: integer ~~ unknown
LINE 1: ...ELECT * FROM thuc_1148_annagnps_cell_ids WHERE dn LIKE '%4' ...
                                                             ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.

[SQL: SELECT * FROM thuc_1148_annagnps_cell_ids WHERE dn LIKE '%%4' LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [17]:
import pandas as pd
from sqlalchemy import URL, create_engine, text as sql_text

url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

query = 'SELECT * FROM usa_valid_soil_layers_data WHERE "Soil_ID" = 568326'

df = pd.read_sql_query(sql=sql_text(query), con=engine.connect())
df

,Soil_ID,Layer_Number,Layer_Depth,Bulk_Density,Clay_Ratio,Silt_Ratio,Sand_Ratio,Rock_Ratio,Very_Fine_Sand_Ratio,CaCO3_Content,...,Base_Saturation,Unstable_Aggregate_Ratio,pH,Organic_Matter_Ratio,Organic_N_Ratio,Inorganic_N_Ratio,Organic_P_Ratio,Inorganic_P_Ratio,Soil_Structure_Code,Input_Units_Code
